In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

%load_ext autoreload
%autoreload 2

In [2]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes.aio import SearchIndexClient

SEARCH_AI_ENDPOINT = os.getenv('AZURE_SEARCH_AI_ENDPOINT')
SEARCH_AI_API_KEY = os.getenv('AZURE_SEARCH_AI_API_KEY')
SEARCH_AI_INDEX_NAME = os.getenv('AZURE_SEARCH_AI_INDEX_NAME')

# Inicializa el cliente
search_client = SearchIndexClient(
    endpoint=SEARCH_AI_ENDPOINT,
    credential=AzureKeyCredential(SEARCH_AI_API_KEY)
)

# Listar índices
indices = [idx.name async for idx in search_client.list_indexes()]
print(f"Índices encontrados: {indices}\n")

doc_client = SearchClient(
    endpoint=SEARCH_AI_ENDPOINT,
    index_name=indices[0],
    credential=AzureKeyCredential(SEARCH_AI_API_KEY)
)

# Buscar los primeros 5 documentos    
doc_search = doc_client.search(
    search_text="*",
    top=34
)

# Almacenar los documentos en una lista
docs = list(doc_search)

Índices encontrados: ['pdf-index']



In [9]:
from openai import AzureOpenAI
from IPython.display import display, HTML
import markdown

OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')

llm = AzureOpenAI(
    api_key=OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
    azure_endpoint=OPENAI_ENDPOINT
)

def pregunta_llm(contexto: str, pregunta: str) -> str:
    messages = [
        {"role": "system", "content": "Eres un asistente experto en búsqueda de información."},
        {"role": "user", "content": f"Usa el siguiente contexto para responder:\n\n{contexto}\n\nPregunta: {pregunta}"}
    ]
    resp = llm.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=messages,
        temperature=0,
        max_tokens=777
    )
    return resp.choices[0].message.content


def generar_respuestas_llm(preguntas: list, docs: list) -> list:
    historial_chat = []
    for doc in docs:
        contexto = doc.get('content')
        if contexto:
            for pregunta in preguntas:
                respuesta_ia = pregunta_llm(contexto, pregunta)
                historial_chat.append({
                    "pregunta": pregunta,
                    "respuesta": respuesta_ia
                })
            break
    return historial_chat

def formatear_chat_html(historial: list) -> str:
    """
    Toma una lista de intercambios de chat y la formatea en HTML
    con un tema oscuro y colores de texto suaves.
    """
    html = """
    <head>
        <style>
            .chat-container { 
                font-family: 'Segoe UI', Arial, sans-serif; 
                border: 1px solid #444; 
                border-radius: 10px; 
                padding: 15px; 
                max-width: 800px; 
                margin: auto; 
                background-color: #1e1e1e; /* Fondo principal oscuro */
            }
            .mensaje { 
                margin-bottom: 15px; 
                padding: 12px; 
                border-radius: 8px; 
                line-height: 1.6; 
                box-shadow: 0 2px 5px rgba(0,0,0,0.2); 
            }
            .mensaje strong { font-weight: bold; }
            
            /* --- Estilo de Usuario (Naranja Oscuro) --- */
            .usuario { 
                background-color: #4a2c00; 
                color: #FFDAB9; /* Color de texto naranja más suave */
                border-left: 5px solid #e65100; 
            }
            
            /* --- Estilo de IA (Azul Oscuro) --- */
            .ia { 
                background-color: #1a2c4b; 
                /* --- CAMBIO CLAVE: Color de texto blanco suave --- */
                color: #EAEAEA; 
                border-left: 5px solid #2196F3; 
            }
            .ia h1, .ia h2, .ia h3 { 
                margin-top: 10px; 
                margin-bottom: 5px; 
                color: #bbdefb; /* Títulos más brillantes */
                border-bottom: 1px solid #444;
                padding-bottom: 4px;
            }
            .ia ul { padding-left: 20px; }
            .ia p { margin: 0; }
            .ia a { color: #82b1ff; text-decoration: none; }
            .ia a:hover { text-decoration: underline; }
            .ia code { background-color: #2b2b2b; padding: 2px 4px; border-radius: 4px; font-family: 'Courier New', Courier, monospace; }
        </style>
    </head>
    <body>
        <div class="chat-container">
    """
    
    for turno in historial:
        pregunta_usuario = turno['pregunta']
        respuesta_ia = turno['respuesta']
        
        # Bloque del Usuario
        html += f'<div class="mensaje usuario">🧑‍💻 <strong>Tú:</strong> {pregunta_usuario}</div>'
        
        # Procesa la respuesta de la IA para convertirla a HTML
        respuesta_formateada = markdown.markdown(respuesta_ia)
        
        # Bloque de la IA
        html += f'<div class="mensaje ia">🤖 <strong>IA:</strong> {respuesta_formateada}</div>'

    html += "</div></body>"
    return html

In [10]:
# --- 3. EJECUCIÓN PRINCIPAL DEL PROCESO ---
# Tu lista de preguntas
preguntas_a_realizar = [
    "¿Cuál es la información principal de este documento?",
    "Haz un resumen y señala el público objetivo del documento.",
    "¿Puedes listar los productos que se encuentran en el documento?",
]

# Generar las respuestas
historial_de_la_conversacion = generar_respuestas_llm(preguntas_a_realizar, docs)

if historial_de_la_conversacion:
    # Formatear el resultado en HTML
    html_output = formatear_chat_html(historial_de_la_conversacion)
    
    # Mostrar el HTML en el notebook
    display(HTML(html_output))
else:
    print("\nNo se generó ninguna conversación. Asegúrate de que 'docs' contenga documentos con el campo 'content'.")